In [461]:
import pandas as pd

import numpy as np



In [462]:
df = pd.read_csv('..df/_data.csv')

In [463]:
df.head(10)
df.shape

(23368, 25)

In [464]:
data = data.drop(['Парковка'], axis=1) # Убераем парковку

In [465]:
data = data.drop(['Телефоны'], axis=1) # Убераем телефоны

In [466]:
data = data.drop(['Площадь комнат, м2'], axis=1) # Убераем площади комнат

In [467]:
temp1=data['Цена']
rent_price_curr = temp1.apply(lambda x:x.split('/')[0].split())
rent_price = rent_price_curr.apply(lambda x:x[0])
# Цена аренды

In [468]:
currency = rent_price_curr.apply(lambda x:x[1]) # Валюта аренды

In [469]:
s3=temp1.apply(lambda x:x.split('Срок аренды - '))
stay_time=s3.apply(lambda x:x[1].split(',')[0]) # Виды сроков

In [470]:
deposite = temp1.apply(lambda x:x.split('Залог - '))
test1 = deposite.apply(lambda x:len(x))
bool_series1 = test1==2

deposite[bool_series1] = deposite[bool_series1].apply(lambda x:x[1].split(',')[0].split(' '))
deposite[~bool_series1] = '00'

deposite_value = deposite.apply(lambda x:x[0]) # Все залоги, если нету = то 0 строкой
deposite_currency = deposite.apply(lambda x:x[1]) # Все валюты, если нету = то 0 строкой


In [471]:
pre_pay = temp1.apply(lambda x:x.split('Предоплата ')[-1])

pre_pay_months = pre_pay.apply(lambda x:x.split(' ')[0]) 

pre_pay_bool = pre_pay.apply(lambda x:x.split(' ')[-1])
pre_pay_bool[(pre_pay_bool=='Длительный')|(pre_pay_bool=='месяцев')]='нету'
# Предоплата либо "мес", либо нету - заменил на "нету".

pre_pay_months = pre_pay_months.astype(float)
pre_pay_months[pre_pay_months>12]=0
# Количество месяцев

In [472]:
temp2=data['Ремонт']
temp2.value_counts(dropna=False)

Ремонт
Косметический    8499
Евроремонт       8470
Дизайнерский     3474
NaN              2755
Без ремонта       170
Name: count, dtype: int64

In [473]:
temp4=data['Балкон']
temp4.value_counts(dropna=False)

Балкон
NaN                       7978
Балкон (1)                7428
Лоджия (1)                6007
Балкон (1), Лоджия (1)     716
Лоджия (2)                 568
Балкон (2)                 474
Балкон (3)                  55
Лоджия (3)                  45
Балкон (2), Лоджия (2)      25
Балкон (1), Лоджия (2)      24
Балкон (2), Лоджия (1)      20
Балкон (4)                   6
Балкон (1), Лоджия (3)       5
Лоджия (4)                   5
Балкон (3), Лоджия (1)       5
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [474]:
data = data.drop(['Цена'], axis=1) # Убераем цену

In [475]:
data.insert(9,'rent_price',rent_price)
data.insert(10,'rent_currency',currency)
data.insert(11,'stay_time',stay_time)
data.insert(12,'deposite',deposite_value)
data.insert(13,'deposite_currency',deposite_currency)
data.insert(14,'is_prepay',pre_pay_bool)
data.insert(15,'prepay_months',pre_pay_months)

In [476]:
data = data.drop(['Описание'], axis=1) # Убераем описание

In [477]:
# data готов = Разобрали Цену, убрали Описание,Парковку, Площадь комнат И Телефон
data.shape

(23368, 27)

In [478]:
data.iloc[:,8:17]

,rent_price,rent_currency,stay_time,deposite,deposite_currency,is_prepay,prepay_months,Ремонт,Балкон
0,500000.0,руб.,Длительный,500000,руб.,мес,1.0,Дизайнерский,NaN
1,500000.0,руб.,Длительный,500000,руб.,мес,1.0,Дизайнерский,NaN
2,500000.0,руб.,Длительный,500000,руб.,мес,1.0,Евроремонт,NaN
3,400000.0,руб.,Длительный,400000,руб.,мес,1.0,Евроремонт,NaN
4,225000.0,руб.,Длительный,225000,руб.,мес,1.0,Евроремонт,NaN
...,...,...,...,...,...,...,...,...,...
23363,42000.0,руб.,Длительный,42000,руб.,мес,1.0,Евроремонт,Балкон (1)
23364,45000.0,руб.,Длительный,45000,руб.,мес,1.0,Евроремонт,Лоджия (1)
23365,50000.0,руб.,Длительный,50000,руб.,мес,1.0,Дизайнерский,Балкон (1)
23366,55000.0,руб.,Длительный,50000,руб.,мес,2.0,Евроремонт,Лоджия (1)


In [479]:
# Здесь начинается 2-ая Стадия, ниже этого коммента, работаю с существующим data и все меняю прямо в нём

In [480]:
data['rent_price']=data['rent_price'].astype(float)
data['deposite']=data['deposite'].astype(float)
# Переводит числовые во float

In [481]:
data['rent_price'].value_counts(dropna=False)

rent_price
40000.0     1555
45000.0     1498
35000.0     1360
50000.0     1294
55000.0      978
            ... 
19999.0        1
62100.0        1
84990.0        1
104500.0       1
64100.0        1
Name: count, Length: 599, dtype: int64

In [482]:
data['deposite'].value_counts(dropna=False) # Если нету депозита, то строка тут 0

deposite
40000.0     1484
45000.0     1378
50000.0     1331
35000.0     1240
0.0          990
            ... 
150.0          1
45600.0        1
34990.0        1
148000.0       1
115990.0       1
Name: count, Length: 556, dtype: int64

In [483]:
data['deposite_currency'].value_counts(dropna=False) # Если нету депозита, то строка тут 0

deposite_currency
руб.    22354
0         990
$          14
€          10
Name: count, dtype: int64

In [529]:
data['deposite_currency'].shape

(23368,)

In [484]:
data['stay_time'].value_counts(dropna=False)

stay_time
Длительный              22305
На несколько месяцев     1063
Name: count, dtype: int64

In [485]:
data = data.drop(['is_prepay'], axis=1) # Убераем логический признак
data['prepay_months'] = data['prepay_months'].astype(int) # Теперь если 0 - то нету Предоплаты, или количество месяцев предоплаты
data['prepay_months'].value_counts() 


prepay_months
1     22489
0       591
2       270
3        11
12        5
11        1
6         1
Name: count, dtype: int64

In [486]:
data = data.rename(columns={'Ремонт': 'renovation'}) # Переименовываем колонку

In [487]:
data['renovation']=data['renovation'].fillna('Нету') # Пропуски = "Нету"

In [488]:
data['renovation'].value_counts(dropna=False)

renovation
Косметический    8499
Евроремонт       8470
Дизайнерский     3474
Нету             2755
Без ремонта       170
Name: count, dtype: int64

In [489]:
data = data.rename(columns={'Балкон': 'balcony'}) # Переименовываем колонку

In [490]:
data['balcony'].dtype

dtype('O')

In [491]:
data['balcony']=data['balcony'].fillna('Балкон (0), Лоджия (0)') # Пропуски = 'Балкон (0), Лоджия (0)'

In [492]:
data['balcony'].value_counts()

balcony
Балкон (0), Лоджия (0)    7978
Балкон (1)                7428
Лоджия (1)                6007
Балкон (1), Лоджия (1)     716
Лоджия (2)                 568
Балкон (2)                 474
Балкон (3)                  55
Лоджия (3)                  45
Балкон (2), Лоджия (2)      25
Балкон (1), Лоджия (2)      24
Балкон (2), Лоджия (1)      20
Балкон (4)                   6
Балкон (1), Лоджия (3)       5
Лоджия (4)                   5
Балкон (3), Лоджия (1)       5
Балкон (2), Лоджия (3)       3
Балкон (1), Лоджия (4)       2
Балкон (3), Лоджия (3)       1
Балкон (4), Лоджия (4)       1
Name: count, dtype: int64

In [493]:
data.iloc[:,8:16]

,rent_price,rent_currency,stay_time,deposite,deposite_currency,prepay_months,renovation,balcony
0,500000.0,руб.,Длительный,500000.0,руб.,1,Дизайнерский,"Балкон (0), Лоджия (0)"
1,500000.0,руб.,Длительный,500000.0,руб.,1,Дизайнерский,"Балкон (0), Лоджия (0)"
2,500000.0,руб.,Длительный,500000.0,руб.,1,Евроремонт,"Балкон (0), Лоджия (0)"
3,400000.0,руб.,Длительный,400000.0,руб.,1,Евроремонт,"Балкон (0), Лоджия (0)"
4,225000.0,руб.,Длительный,225000.0,руб.,1,Евроремонт,"Балкон (0), Лоджия (0)"
...,...,...,...,...,...,...,...,...
23363,42000.0,руб.,Длительный,42000.0,руб.,1,Евроремонт,Балкон (1)
23364,45000.0,руб.,Длительный,45000.0,руб.,1,Евроремонт,Лоджия (1)
23365,50000.0,руб.,Длительный,50000.0,руб.,1,Дизайнерский,Балкон (1)
23366,55000.0,руб.,Длительный,50000.0,руб.,2,Евроремонт,Лоджия (1)
